In [1]:
%reload_ext watermark
%watermark -a "Hugo Mendes"

Author: Hugo Mendes



# Houses Prediction

## Objetivo: Gerar um modelo de previsão a partir do dataset escolhido Kaggle para inferir novos valores para casas que serão passadas pela aplicação de API do projeto.
## Etapas: Vamos utilizar de todas as etapas de um projeto Data Science (Carregando dos dados, Conhecimento dos dados, Análise Exploratória, Limpeza e Pré Processamento, Geração do Modelo, Validações e Avaliações, Importar o modelo para o deploy (produção).
## Metas: Pretendo alcançar um modelo de previsão com o poder de explicabilidade maior que 85% e que o modelo seja estatisticamente significativo e também que minimizamos ao máximo o MAE (erro absoluto médio) e RMSE (raiz do erro quadrático médio).

### 1. Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
# versões dos pacotes
%watermark --iversions

### 2. Carregando os dados

In [ ]:
dataset = "Housing.csv"
df_casas = pd.read_csv(dataset, sep=",")

In [ ]:
# Exibindo linhas inicias do DataFrame
df_casas.head()

In [ ]:
# Exibindo linhas finais do Dataframe
df_casas.tail()

### 3. Conhecendo os dados

In [ ]:
# Informações do Dataframe
df_casas.info()

In [ ]:
# Quantidade de atributos (colunas)
len(df_casas.columns)

In [ ]:
# Quantidade de linhas
df_casas.shape[0]

In [ ]:
# Quantidade de registros
df_casas.shape[0] * df_casas.shape[1]

In [ ]:
# Tipos de dados
df_casas.dtypes

In [ ]:
df_casas.select_dtypes(include=np.number).columns # Colunas numéricas

In [ ]:
df_casas.select_dtypes(include="object").columns # Colunas categoricas

### 4. Análise Exploratória

In [ ]:
# Valores ausentes
df_casas.loc[df_casas.isnull().any(axis=1)]

In [ ]:
df_casas.isnull().sum().sum()

In [ ]:
# Linhas duplicadas
df_casas[df_casas.duplicated()]

In [ ]:
df_casas[df_casas.duplicated(subset=df_casas.drop(columns="price").columns, keep=False)].sort_values(by="area", ascending=False)

In [ ]:
media_casa_1 = np.mean(df_casas.loc[[408,496], "price"]) # média do preço
df_casas.loc[408, "price"] = media_casa_1 # substituindo pela média
df_casas.drop(496, axis=0, inplace=True) # removendo a outra linha

In [ ]:
media_casa_2 = np.mean(df_casas.loc[[458,544], "price"])
df_casas.loc[458, "price"] = media_casa_2
df_casas.drop(544, axis=0, inplace=True) 

In [ ]:
media_casa_3 = np.mean(df_casas.loc[[385,525], "price"])
df_casas.loc[385, "price"] = media_casa_3
df_casas.drop(525, axis=0, inplace=True) 

In [ ]:
df_casas[df_casas.duplicated(subset=df_casas.drop(columns="price").columns, keep=False)].sort_values(by="area", ascending=False)

In [ ]:
# Medidas estatisticas nas colunas numéricas
df_casas.describe(include=np.number)

In [ ]:
# Medidas estatisticas nas colunas categoricas
df_casas.describe(include="object")

In [ ]:
# Calculando o coeficiente de variação das variáveis numéricas
for column in df_casas.select_dtypes(include=np.number).columns:
    cv = (df_casas[column].std() / df_casas[column].mean()) * 100
    print(f"A variável {column} tem um coeficiente de variação de {cv:.2f}%")

In [ ]:
# Distribuição dos atributos
plt.figure(figsize=(12,6))
sns.histplot(df_casas["price"], kde=True)
plt.axvline(df_casas["price"].mean(), color = "red", linestyle = '--', linewidth = 2, label = f"Média")
plt.axvline(df_casas["price"].median(), color = "orange", linestyle = '--', linewidth = 2, label = f"Mediana")
plt.title("Histograma da variável Preço")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(df_casas["area"], kde=True)
plt.axvline(df_casas["area"].mean(), color = "red", linestyle = '--', linewidth = 2, label = f"Média")
plt.axvline(df_casas["area"].median(), color = "orange", linestyle = '--', linewidth = 2, label = f"Mediana")
plt.legend()
plt.title("Histograma da variável Area")
plt.show()

In [ ]:
colunas_distribuicao = [
    "bedrooms", "bathrooms", "stories", "parking", 
    "mainroad", "guestroom", "basement", "hotwaterheating",
    "airconditioning", "prefarea", "furnishingstatus"
]

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(20, 20))
plt.subplots_adjust(hspace=0.4, wspace=0.3)
for i, coluna in enumerate(colunas_distribuicao):
    plt.subplot(4, 3, i + 1) # Cria o grid 4x3
    
    # Criando o countplot
    ax = sns.countplot(data=df_casas, x=coluna, palette="viridis", hue=coluna, legend=False)
    
    # Customização de cada gráfico
    plt.title(f"Distribuição da variável {coluna.capitalize()}", fontsize=14, fontweight="bold")
    plt.xlabel("") 
    plt.ylabel("Quantidade")
plt.show()

In [ ]:
# Detecção de outliers
df_casas.drop(columns=["price","area"]).select_dtypes(include=np.number).boxplot(figsize=(12,6))
plt.title("BoxPlots das variáveis")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(df_casas["price"])
plt.title("BoxPlot da variável Preço")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(df_casas["area"])
plt.title("BoxPlot da variável Area")
plt.show()

### 5. Limpeza e Pré Processamento

In [ ]:
df_casas.columns

In [ ]:
# Renomeando as colunas para português
colunas_pt = ["preco", "area", "quartos", "banheiros", "andares", "acesso_rodovia",
             "quarto_hospede", "porao", "aquecimento_agua", "ar_condicionado", "vagas_estacionameno", 
             "area_preferencial", "status_mobilia"]

In [ ]:
colunas_df_pt = {coluna_df: coluna_br for coluna_df, coluna_br in zip(df_casas.columns, colunas_pt)} # dicionário para renomear as colunas

In [ ]:
df_limpo = df_casas.rename(columns=colunas_df_pt) # novo df para o pré processamento (com as colunas em portugues)

In [ ]:
# Tratanto outliers da variável preço
IC = df_limpo["preco"].quantile(0.75) - df_limpo["preco"].quantile(0.25) # tamanho da caixa do boxplot
formula_preco = df_limpo["preco"].quantile(0.75) + (1.5 * IC) # formula para encontrar valores extremos a partir do ponto máximo

In [ ]:
# Visualizando os registros com outliers em preço
outliers_preco = df_limpo[df_limpo["preco"] > formula_preco]

In [ ]:
outliers_preco

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(data=outliers_preco, x="area_preferencial")
plt.ylabel("Quantidade")
plt.title("Outliers preço por area preferencial")
plt.show()

In [ ]:
# Tratando outliers da variável area
# Antes disso vamos passar a area de pés quadrados para metros quadrados
df_limpo["area"] = df_limpo["area"] * 0.0929

In [ ]:
IC2 = df_limpo["area"].quantile(0.75) - df_limpo["area"].quantile(0.25) # tamanho da caixa do boxplot
formula_area = df_limpo["area"].quantile(0.75) + (1.5 * IC2) # formula para encontrar valores extremos a partir do ponto máximo

In [ ]:
outliers_area = df_limpo[df_limpo["area"] > formula_area]

In [ ]:
outliers_area

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=df_limpo,x="area", hue="quartos")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=df_limpo,x="area", hue="banheiros")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=df_limpo,x="area", hue="andares")
plt.show()

In [ ]:
df_limpo

In [ ]:
mapeamento = {
    "no":0,
    "yes":1
}

In [ ]:
for coluna in df_limpo.select_dtypes(include=["object"]).drop(columns="status_mobilia").columns:
    df_limpo[coluna] = df_limpo[coluna].map(mapeamento)

In [ ]:
"""
Mapeamento de
0 = no
1 = yes

nas variáveis (acesso_rodovia, quarto_hospede, porao, aquecimento_agua,
       ar_condicionado, area_preferencial)
"""
df_limpo # depois do mapeamento

In [ ]:
# Agora vamos mapear a variável status_mobilia para passar pro portugues
mapeamento_mobilia = {
    "furnished":"mobiliada",
    "semi-furnished":"semi-mobiliada",
    "unfurnished":"sem-mobilia"
}

In [ ]:
df_limpo["status_mobilia"] = df_limpo["status_mobilia"].map(mapeamento_mobilia)

In [ ]:
df_limpo

In [ ]:
df_limpo = pd.get_dummies(df_limpo, columns=["status_mobilia"], dtype=int)

In [ ]:
df_limpo

In [ ]:
df_limpo.dtypes

In [ ]:
colunas_binarias = ["acesso_rodovia", "quarto_hospede", "porao", "aquecimento_agua",
       "ar_condicionado", "area_preferencial", "status_mobilia_mobiliada", "status_mobilia_sem-mobilia", 
        "status_mobilia_semi-mobiliada"]

In [ ]:
normalizar = MinMaxScaler()
normalizar.fit(df_limpo.drop(columns=colunas_binarias))
colunas_normalizadas = pd.DataFrame(data=normalizar.transform(df_limpo.drop(columns=colunas_binarias)), 
                                    columns=df_limpo.drop(columns=colunas_binarias).columns)

In [ ]:
colunas_normalizadas # Dados normalizados

In [ ]:
round(colunas_normalizadas.describe(),2) # Minimo 0 e Máximo 1

In [ ]:
padronizar = StandardScaler()
padronizar.fit(df_limpo.drop(columns=colunas_binarias))
colunas_padronizadas = pd.DataFrame(data=padronizar.transform(df_limpo.drop(columns=colunas_binarias)), 
                                    columns=df_limpo.drop(columns=colunas_binarias).columns)

In [ ]:
colunas_padronizadas # Dados padronizados

In [ ]:
round(colunas_padronizadas.describe(),2) # Média 0 e Desvio Padrão 1

In [ ]:
# Dataframe limpo e normalizado
df_limpo_normalizado = pd.concat([colunas_normalizadas, df_limpo.drop(columns=colunas_normalizadas.columns)], axis=1)

In [ ]:
df_limpo_normalizado.head()

In [ ]:
# Dataframe limpo e padronizado
df_limpo_padronizado = pd.concat([colunas_padronizadas, df_limpo.drop(columns=colunas_padronizadas.columns)], axis=1)

In [ ]:
df_limpo_padronizado.head()

### 6. Modelo

In [ ]:
for coluna in df_casas.select_dtypes(include=np.number):
    teste = shapiro(df_casas[coluna])
    if teste.pvalue > 0.05:
        print(f"Nossa variável {coluna} assume uma distribuição normal")
    else:
        print(f"Nossa variável {coluna} não assume uma distribuição normal")

In [ ]:
df_casas.hist(bins=30, figsize=(12,6))
plt.tight_layout()
plt.show()

In [ ]:
corr_matrix = df_limpo.corr(method="spearman")

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=.5)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.scatterplot(x=df_limpo["preco"],y= df_limpo["area"])
plt.title("Gráfico de dispersão area x preço")
plt.tight_layout()
plt.show()

In [ ]:
# Dividindo conjunto de treino e teste
X = df_limpo_padronizado.drop(columns="preco").values
Y = df_limpo_padronizado["preco"].values